In [1]:
from dataclasses import dataclass

#defining the classes that will act as objects for fights and record

@dataclass
class Fight:
    R_fighter:str
    B_fighter:str
    R_record:list
    B_record:list
    Rematch:bool
    R_KD:int
    B_KD:int
    R_SIG_STR:list  #need to split
    B_SIG_STR:list  #need to split
    R_TOTAL_STR:list    #need to split
    B_TOTAL_STR:list    #need to split
    R_TD:int
    B_TD:int
    R_SUB_ATT:int
    B_SUB_ATT:int
    R_CTRL:int  #convert to only seconds
    B_CTRL:int  #convert to only seconds
    R_HEAD:list #need to split
    B_HEAD:list #need to split
    win_by:str
    last_round:int
    last_round_time:int #convert to only seconds
    Format:str  
    date:str
    Fight_type:str #Format change
    Winner:str
    
@dataclass
class Record:
    fighter_name: str
    record: list #ordered as wins, losses draws; type:int


In [2]:
#open the first database and load it into excel

f = open("ufc_database_1993_2023_python.csv", "r")
excel = f.read().split("\n")

header = excel[0]
print(header)
header = header.split(',')
excel = excel[1:7273] #the index  was 7273
#5133 is the index you should put if you want
#fight data for 10 years (2013-2023)
f.close()
print("fight database successfully imported")

#open the second database and read it into record

f = open("ufc-fighters-statistics.csv", "r")
record = f.read().split("\n")
header2 = record[0].split(",")
record = record[1:4110]   #this now takes the whole database
f.close()
print("record database successfully imported")



R_fighter,B_fighter,R_record,B_record,Rematch,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_TOTAL_STR.,B_TOTAL_STR.,R_TD,B_TD,R_SUB_ATT,B_SUB_ATT,R_CTRL,B_CTRL,R_HEAD,B_HEAD,win_by,last_round,last_round_time,Format,date,Fight_type,Winner
fight database successfully imported
record database successfully imported


In [3]:
# a function that converts 1:03 into 63 (Time into seconds only format)
def seconds(time):
    """time is a string
    the return type for this function is int"""
    time = time.split(":")
    seconds = int(time[0])*60+ int(time[1])
    return seconds


database = []
for datapoint in excel:
    datapoint = datapoint.split(",")
    fight = Fight(datapoint[0], datapoint[1], datapoint[2], datapoint[3], datapoint[4]
                  , datapoint[5], datapoint[6], datapoint[7].split(" of "), datapoint[8].split(" of "), 
                  datapoint[9].split(" of "), datapoint[10].split(" of "), 
                  datapoint[11], datapoint[12], datapoint[13], datapoint[14],
                    seconds(datapoint[15]), seconds(datapoint[16]), datapoint[17].split(" of "), 
                    datapoint[18].split(" of "), datapoint[19], datapoint[20], seconds(datapoint[21]),
                    datapoint[22], datapoint[23], datapoint[24], datapoint[25])
    database.append(fight)

print(len(database), len(excel))
print(database[7271])


#this is the part where I turn record into a class...

ufcFighters = []
for fighter in record:
    items = fighter.split(",")
    fightrecord = [int(items[1]),int(items[2]),int(items[3])]
    who = Record(items[0], fightrecord)
    ufcFighters.append(who)   

print(len(ufcFighters), len(record))
print(ufcFighters[4108])

#This code block uses both excel[] to produce database which is a list of objects(fights)
#that are properly formatted for analysis
#Then it uses record[] to produce ufcFighters which is a list of objects(fighter name and record)
#that are properly formatted for analysis

7272 7272
Fight(R_fighter='Scott Morris', B_fighter='Sean Daugherty', R_record='', B_record='', Rematch='', R_KD='0', B_KD='0', R_SIG_STR=['1', '1'], B_SIG_STR=['0', '4'], R_TOTAL_STR=['2', '2'], B_TOTAL_STR=['1', '5'], R_TD='1 of 1', B_TD='0 of 0', R_SUB_ATT='1', B_SUB_ATT='0', R_CTRL=0, B_CTRL=0, R_HEAD=['1', '1'], B_HEAD=['0', '2'], win_by='Submission', last_round='1', last_round_time=20, Format='No Time Limit', date='11-Mar-94', Fight_type='Open Weight Bout', Winner='Scott Morris')
4109 4109
Record(fighter_name='Shaun Etchell', record=[7, 3, 0])


In [4]:
#This is the part where I sync fighter names and records
from copy import deepcopy

#wow, a function that returns the fighter's record
def recordmanip(name, manip):
    """This function gets a fighters name,
    and a int manip (0: fighter won the match, 1: fighter lost the match, 2: fighter tie)
    and uses both these to find the fighter's record, remove one win or loss from
    his record and return the edited record"""
    for fighter in ufcFighters:
        if fighter.fighter_name == name:
            fighter.record[manip] -=1
            break 
    return fighter.record
    #this function is ready


newdatabase = []
for fight in database:
    fighter1 = fight.R_fighter
    fighter2 = fight.B_fighter
    winner = fight.Winner
    if (fighter1 == winner):
        #remove one win from fighter 1
        f1record = recordmanip(fighter1, 0)
        #remove one loss from fighter 2
        f2record = recordmanip(fighter2, 1)
    elif(fighter2 == winner):
        #remove one win from fighter 2
        f2record = recordmanip(fighter2, 0)
        #remove one loss from fighter 1
        f1record = recordmanip(fighter1, 1)
    else:
        #check win by
        winby = fight.win_by
        #if it contains decision, remove draw
        if ("Decision" in winby):
            f1record = recordmanip(fighter1, 2)
            f2record = recordmanip(fighter2, 2)
        else:
            #otherwise pass
            pass
    fight.R_record = f1record #this is where the break happens
    fight.B_record = f2record
    what = deepcopy(fight)
    newdatabase.append(what)
    #for some reason the record always changed and was synced for the same 
    #fighter throughout the database, no idea why.
    #so using deepcopy which is "a copy whose properties do not share the same
    #references (point to the same underlying values) as those of the source 
    #object from which the copy was made" fixed the issue.

database = newdatabase
#this turns things back to how they were

In [5]:
#this code block is supposed to tackle fight type...

for fight in database:
    if ("Title" in fight.Fight_type):
        if("Women" in fight.Fight_type):
            fight.Fight_type = "Women's Title Bout"
        else:
            fight.Fight_type = "Men's Title Bout"
    else:
        if("Women" in fight.Fight_type):
            fight.Fight_type = "Women's Bout"
        else:
            fight.Fight_type = "Men's Bout"

#this code works

In [6]:
#this code is supposed to determine whether a fight is a rematch...



for x in range(len(database)):
    #the list format is [Red corner name, Blue corner name]
    fighters = [database[x].R_fighter, database[x].B_fighter]
    fighters = sorted(fighters)
    database[x].Rematch = False
    for i in range(x+1, len(database)):
        otherfighters = [database[i].R_fighter, database[i].B_fighter]
        otherfighters = sorted(otherfighters)
        if(otherfighters == fighters):
            database[x].Rematch = True
            break

#this now works

<h3>What has been done so far: </h3>
 1. Define Classes that will be used to hold row data<br />
 2. Import the files<br />
 3. Organize rows into objects inside of lists<br />
 4. I tackle the issue of the fighter record, and fix it<br />
 5. I optimize fight type to only contain relevant information<br />
 6. I find rematches and set their bool to True<br />
 7. (Upcoming) Output the lists into a new csv

In [9]:
#This code block will put the database into a csv...

f = open("Complete_database.csv", "w")
#what we want is f.write(something)

#headers:
f.write("R_fighter,B_fighter,R_record,B_record,Rematch,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_TOTAL_STR.,B_TOTAL_STR.,R_TD,B_TD,R_SUB_ATT,B_SUB_ATT,R_CTRL,B_CTRL,R_HEAD,B_HEAD,win_by,last_round,last_round_time,Format,date,Fight_type,Winner")
f.write("\n")

#writing the data:
for fight in database:
    #fix the formatting to save effort later
    f.write(fight.R_fighter)
    f.write(",")
    f.write(fight.B_fighter)
    f.write(",")
    f.write(str(fight.R_record[0]) + "_" + str(fight.R_record[1]) + "_" + str(fight.R_record[2]))
    f.write(",")
    f.write(str(fight.B_record[0]) + "_" + str(fight.B_record[1]) + "_" + str(fight.B_record[2]))
    f.write(",")
    f.write(str(fight.Rematch))
    f.write(",")
    f.write(str(fight.R_KD))
    f.write(",")
    f.write(str(fight.B_KD))
    f.write(",")
    f.write(fight.R_SIG_STR[0] + "_" + fight.R_SIG_STR[1])
    f.write(",")
    f.write(fight.B_SIG_STR[0] + "_" +fight.B_SIG_STR[1])
    f.write(",")
    f.write(fight.R_TOTAL_STR[0] + "_" +fight.R_TOTAL_STR[1])
    f.write(",")
    f.write(fight.B_TOTAL_STR[0] + "_" +fight.B_TOTAL_STR[1])
    f.write(",")
    f.write(str(fight.R_TD))
    f.write(",")
    f.write(str(fight.B_TD))
    f.write(",")
    f.write(str(fight.R_SUB_ATT))
    f.write(",")
    f.write(str(fight.B_SUB_ATT))
    f.write(",")
    f.write(str(fight.R_CTRL))
    f.write(",")
    f.write(str(fight.B_CTRL))
    f.write(",")
    f.write(fight.R_HEAD[0] + "_" + fight.R_HEAD[1])
    f.write(",")
    f.write(fight.B_HEAD[0] + "_" + fight.B_HEAD[1])
    f.write(",")
    f.write(fight.win_by)
    f.write(",")
    f.write(str(fight.last_round))
    f.write(",")
    f.write(str(fight.last_round_time))
    f.write(",")
    f.write(fight.Format)
    f.write(",")
    f.write(fight.date)
    f.write(",")
    f.write(fight.Fight_type)
    f.write(",")
    f.write(fight.Winner)
    f.write("\n")

f.close()
print("Complete.")

#This is done, but we are not...

Complete.


In [7]:
#this is was added later, to get the optimized database

f = open("Optimized_database.csv", "w")

#headers:
f.write("R_fighter,B_fighter,R_record,B_record,Rematch,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_TD,B_TD,R_SUB_ATT,B_SUB_ATT,R_CTRL,B_CTRL,R_HEAD,B_HEAD,win_by,last_round,last_round_time,date,Fight_type,Winner")
f.write("\n")

#writing the data:
for fight in database:
    #fix the formatting to save effort later
    f.write(fight.R_fighter)
    f.write(",")
    f.write(fight.B_fighter)
    f.write(",")
    f.write(str(fight.R_record[0]) + "_" + str(fight.R_record[1]) + "_" + str(fight.R_record[2]))
    f.write(",")
    f.write(str(fight.B_record[0]) + "_" + str(fight.B_record[1]) + "_" + str(fight.B_record[2]))
    f.write(",")
    f.write(str(fight.Rematch))
    f.write(",")
    f.write(str(fight.R_KD))
    f.write(",")
    f.write(str(fight.B_KD))
    f.write(",")
    f.write(fight.R_SIG_STR[0] + "_" + fight.R_SIG_STR[1])
    f.write(",")
    f.write(fight.B_SIG_STR[0] + "_" +fight.B_SIG_STR[1])
    f.write(",")
    f.write(str(fight.R_TD))
    f.write(",")
    f.write(str(fight.B_TD))
    f.write(",")
    f.write(str(fight.R_SUB_ATT))
    f.write(",")
    f.write(str(fight.B_SUB_ATT))
    f.write(",")
    f.write(str(fight.R_CTRL))
    f.write(",")
    f.write(str(fight.B_CTRL))
    f.write(",")
    f.write(fight.R_HEAD[0] + "_" + fight.R_HEAD[1])
    f.write(",")
    f.write(fight.B_HEAD[0] + "_" + fight.B_HEAD[1])
    f.write(",")
    f.write(fight.win_by)
    f.write(",")
    f.write(str(fight.last_round))
    f.write(",")
    f.write(str(fight.last_round_time))
    f.write(",")
    f.write(fight.date)
    f.write(",")
    f.write(fight.Fight_type)
    f.write(",")
    f.write(fight.Winner)
    f.write("\n")

f.close()
print("Complete.")


Complete.
